<a href="https://colab.research.google.com/github/eusebiogerman/WEBSCRAPING/blob/master/nba_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import unittest
import time
import sys
import os
from os import path
import traceback
import requests
import logging

from console_progressbar import ProgressBar
from bs4 import BeautifulSoup , Comment 
from lxml import etree

#test
def newtag(bs : BeautifulSoup, tagn : str ,text: str) :
    tag = bs.new_tag(tagn)
    tag.string = text 
    return tag

def get_trace_value(plist):
    trace = ''
    for p in plist:
        trace = trace + p + '/n'
    return trace

def load_nba_schedule(year):
    
    #BASE_URL = "https://www.baseball-reference.com" 
    BASE_URL = "https://www.basketball-reference.com/"
    link = f"{BASE_URL}/leagues/NBA_{year}_games.html"
    link2 = ''
    archivo = ''
    fecha = ''
    schedule = []
    gdir = str(year)
    dummycnt = 1
    
    try:
        #crear el repots
        if not (path.isdir('sessons')):
            os.mkdir('sessons')

        #carga el archivo de los datos
        #file = open("mlb_game.txt","a")

        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
        
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            print(f"Schedule {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')
            div_schedule = soup.find_all('div', {'id','div_schedule'})
            table = soup.find_all('table')
            tb = table[0].find_all('tbody')
            trs = tb[0].find_all('tr')

            for tr in trs:
                tds = tr.find_all('td', {'class','center'})
                for td in tds:
                    a = td.find_all('a')
                    if (a) :
                       dict = {}
                       fecha = a[0].get('href').split('/')[2][0:9]
                       archivo = f"sessons/{gdir}.txt"
                       link2 =  f"{BASE_URL}{a[0].get('href')}"
                       dict ={'link' : link2, 'archivo' : archivo, 'fecha' : fecha }  
                       schedule.append(dict)
        return schedule
    except ValueError:
            print("Error de Valor")
            traceback.print_exception(*sys.exc_info())
            return schedule
    except:    
            print("Error de general") 
            traceback.print_exception(*sys.exc_info())   
            return schedule                     


def load_nba_games(link, archivo, gamedate):
    
    logfile = archivo.split('/')[1].replace('txt','log')
    logfile = archivo.split('/')[0] +'/'+ logfile
    logging.basicConfig(filename = 'test.log', enconding ='utf-8',level=logging.INFO)
    plays = [] 

    try:

        #carga el archivo de los datos
        #file = open(archivo, 'a')

        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
        
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            logging.info(f"Process {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')

            #tag escondido para listar los equipos
            comments = soup.find_all(text=lambda text:isinstance(text, Comment))
            htm = str([htm for htm in comments if "div_other_scores" in htm]);

            htmwrap = BeautifulSoup(htm,'html.parser')
            te_as =  htmwrap.find_all('div',class_ ='game_summary nohover current')[0].find_all('a')
            teams = [teams.text for teams in te_as]
            del teams[1]
 
            #codigo de los equipos
            div_qs = soup.find_all('div',class_='filter switcher')
            divbox = div_qs[0].find_all('div')
            divbox = [x.a.get('data-show').split(',') for x in divbox]
            del divbox[0]
            #print(divbox)  
            for box_out in divbox:
                #print(box_out)
                for box in box_out:
                    #print(f"div_{box[1:]}-basic")
                    qter = box[-1]
                    divb = f"{box[1:]}-basic"
                    #print(f"div_{divb}")
                    dbx = soup.find_all('div',{'id':f"div_{divb}"})
                    tb   = dbx[1].find_all('table',{'id':divb})
                    tbo  = tb[0].find_all('tbody')
                    trs  = tbo[0].find_all('tr')
                    for tr in trs :
                        player = tr.find_all('th')[0].text.upper()
                        #print(player)
                        tds  = tr.find_all('td')
                        if (len(tds) > 1):
                            tds  = [" ".join(x.text.replace(',','|').upper().split()) for x in tds]
                            
                            tds.insert(0,int(tds[0].replace(':','')))
                            tds.insert(0,qter)
                            tds.insert(0,player)
                            plays.append(tds)
            plays = sorted(plays, key = lambda x :(x[0],x[1]))  
            for y in plays:
                del y[1] 

            return plays
    except ValueError:
            #file.close 
            print("Error de Valor : load_mlb_games ")
            logging.error(get_trace_value(sys.exc_info()))
    except:    
            #file.close                     
            print("Error de general : load_mlb_games") 
            logging.error(get_trace_value(sys.exc_info()))




'''
sche = load_nba_schedule(2018) 
for es in sche:
    print(es)
'''
score  = {'link': 'https://www.basketball-reference.com//boxscores/201710230DEN.html', 'archivo': 'sessons/2018.txt', 'fecha': '201710230'}
pl =  load_nba_games(score['link'],score['archivo'],score['fecha'])
print(pl) 

# https://www.basketball-reference.com/leagues/NBA_2018_games.html
# https://www.basketball-reference.com/teams/DEN/2021_games.html
# https://www.basketball-reference.com/boxscores/202012230POR.html

 





[['BRADLEY BEAL', 854, '8:54', '1', '3', '.333', '1', '3', '.333', '0', '0', '', '0', '1', '1', '2', '0', '0', '0', '0', '3', '-5'], ['BRADLEY BEAL', 1805, '18:05', '3', '7', '.429', '1', '3', '.333', '2', '2', '1.000', '0', '1', '1', '2', '0', '0', '0', '1', '9', '0'], ['BRADLEY BEAL', 911, '9:11', '2', '4', '.500', '0', '0', '', '2', '2', '1.000', '0', '0', '0', '0', '0', '0', '0', '1', '6', '+5'], ['BRADLEY BEAL', 1709, '17:09', '5', '10', '.500', '0', '2', '.000', '1', '1', '1.000', '1', '3', '4', '0', '1', '0', '2', '0', '11', '+5'], ['BRADLEY BEAL', 847, '8:47', '3', '6', '.500', '0', '2', '.000', '0', '0', '', '1', '0', '1', '0', '0', '0', '0', '0', '6', '+5'], ['BRADLEY BEAL', 822, '8:22', '2', '4', '.500', '0', '0', '', '1', '1', '1.000', '0', '3', '3', '0', '1', '0', '2', '0', '5', '0'], ['EMMANUEL MUDIAY', 329, '3:29', '1', '1', '1.000', '0', '0', '', '1', '2', '.500', '0', '1', '1', '0', '0', '0', '0', '0', '3', '-3'], ['EMMANUEL MUDIAY', 808, '8:08', '2', '3', '.667', '0',

In [30]:
test_list = [['Rash', 4, 28], ['Varsha', 2, 20], ['Nikhil', 1, 20], ['Akshat', 3, 21]] 
print ("The original list is : " + str(test_list)) 
test_list.sort(key = lambda test_list: test_list[1]) 
print ("List after sorting by 2nd element of lists : " + str(test_list)) 

The original list is : [['Rash', 4, 28], ['Varsha', 2, 20], ['Nikhil', 1, 20], ['Akshat', 3, 21]]
List after sorting by 2nd element of lists : [['Nikhil', 1, 20], ['Varsha', 2, 20], ['Akshat', 3, 21], ['Rash', 4, 28]]
